# How we can make projects using Langchain & Hugging Face Open source models

#01. INSTALLATION

In [ ]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


# 02 . **Setup and Environment**

In [ ]:
import os

In [ ]:
# Addition of Hugging face API key

os.environ["HUGGINGFACEHUB_API_TOKEN"] = " " # token is deleted due to security purpose

# 03. Use of LLM

In [ ]:
!pip install huggingface_hub # install huggingface hub library

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
# add the model want to use -> using API KEY

# temperature -> creative parameter , max_length -> how many tokens

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

llm.invoke("translate english to german : my name is Anamika")

'meinem Namen Anamika'

In [ ]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})
result = llm.predict("suggest me a startup name for software soliution")
print(result)

<ipython-input-11-eac3ab36acb1>:3: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm.predict("suggest me a startup name for software soliution")


soliution


# 04. Prompt template

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables=['industry'],
    template="suggest me a startup name for {industry}"
)
p = prompt_template_name.format(industry="Artificial Intelligence")
print(p)

suggest me a startup name for Artificial Intelligence


# 05. Chains


- This combine LLMs and Prompts in multistep workflow

In [ ]:
from langchain.chains import LLMChain

# how internally it is creating everything , add verbose parameter

chain = LLMChain(llm=llm, prompt=prompt_template_name , verbose = True)
response = chain.run("Artificial Intelligence")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
suggest me a startup name for Artificial Intelligence

> Finished chain.
ai


# Simple Sequential chains

- this will show only the last output not each output for each prompt

In [ ]:
llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.5, "max_length":64})

# prompt 1
prompt_template_name = PromptTemplate(
    input_variables=['industry'],
    template="suggest me a startup name for {industry}"
)
startup_name = LLMChain(llm=llm, prompt=prompt_template_name , verbose = True)

# prompt 2
prompt_template_roles = PromptTemplate(
    input_variables=['job roles'],
    template="suggest me some job roles for {industry}"
)

job_roles = LLMChain(llm=llm, prompt=prompt_template_roles , verbose = True)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[startup_name, job_roles])
response = chain.run("Artificial Intelligence")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
suggest me a startup name for Artificial Intelligence

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some job roles for ai

> Finished chain.
Architect


# Sequential chain

In [ ]:
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.chains import SequentialChain

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.5, "max_length":64})

# prompt 1
prompt_template_name = PromptTemplate(
    input_variables=['industry'],
    template="suggest me a startup name for {industry}"
)
# output key for the first chain
startup_name = LLMChain(llm=llm, prompt=prompt_template_name, output_key="startup_name", verbose=True)

# prompt 2
prompt_template_roles = PromptTemplate(
    input_variables=['industry'], # Use 'industry' here instead of 'job roles'
    template="suggest me some job roles for {industry}"
)

#  output key for the second chain
job_roles = LLMChain(llm=llm, prompt=prompt_template_roles, output_key="job_roles", verbose=True)

chain = SequentialChain(
    chains=[startup_name, job_roles],
    input_variables=['industry'],
    output_variables=['startup_name', 'job_roles'],
    verbose=True
)

In [ ]:
# accessing the outputs using correct keys
response = chain({'industry': "Artificial Intelligence"})
startup_name_result = response['startup_name']
job_roles_result = response['job_roles']

print(f"Startup Name: {startup_name_result}")
print(f"Job Roles: {job_roles_result}")



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
suggest me a startup name for Artificial Intelligence

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some job roles for Artificial Intelligence

> Finished chain.

> Finished chain.
Startup Name: ai
Job Roles: Artificial Intelligence (AI) is a field of research that focuses on the development of artificial intelligence systems that can be used to solve problems in the field of computer science.


# 06. Agents and Tools

- Help to plugin with different tools

In [ ]:
# install the package want to use
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=4c5594b5776dcef1073b22944d7a134dc04fc572f84eb1e67d2762a39b8eab5f
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import HuggingFaceHub

In [ ]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("What was the GDP of US in 2024?")

# 07. Memory

- Conversation Buffer memory -> it will remember all the conversation

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True, memory=memory)
name = chain.run("mexican")
print(name)

<ipython-input-56-f5fa6b2620c3>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




> Entering new LLMChain chain...
Prompt after formatting:
suggest me a startup name for mexican

> Finished chain.
el agua


In [ ]:
name = chain.run("arabic")
print(name)



> Entering new LLMChain chain...
Prompt after formatting:
suggest me a startup name for arabic

> Finished chain.
Arabic


In [ ]:
print(chain.memory.buffer)

Human: mexican
AI: el agua
Human: indian
AI: elijah
Human: arabic
AI: Arabic


- Coversation Chain -> just remember few past conversation


In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm, verbose=True, memory=memory)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<ipython-input-60-d0e37ec684a4>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=llm, verbose=True, memory=memory)


In [ ]:
convo.run("how won the first cricket world cup")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: mexican
AI: el agua
Human: indian
AI: elijah
Human: arabic
AI: Arabic
Human: how won the first cricket world cup
AI:

> Finished chain.


'India won the first cricket world cup in 1966'

# 08. Document Loaders

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.4 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/season-12.pdf")
pages = loader.load()

In [ ]:
pages

[Document(metadata={'source': '/content/season-12.pdf', 'page': 0}, page_content='SheFi\nCERTIFICATE OF COMPLETION\nPresented To\nAnamika Ghosh\nGRADUATING FROM\nSeason 12\nDate: \n2024-12-19\nSerial No. 2024\ncert_b17s34q4\n \n')]

- we can use this to make any RAG based application and we can also use this in our projects to give specific kind of answers.